# OPERATION PROGRAMS TO `ORDER`
1. Setup
2. Place naked orders
3. Place cover trades
4. Place orphan trades
5. Delete all naked sells

# 1. Setup

In [1]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    pd.options.display.float_format = '{:,.2f}'.format
    pd.set_option('display.notebook_repr_html', True)
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder 

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_nakeds.pkl', 'df_ohlcs.pkl', 'df_opts.pkl',
       'df_symlots.pkl', 'df_unds.pkl', 'dfrq.pkl', 'qopt_rejects.pkl',
       'qopts.pkl'], dtype='<U16')

# 2. `naked` Orders

In [7]:
# * IMPORTS
from ib_insync import LimitOrder
from support import get_openorders, place_orders

In [8]:
# * RUN nakeds
THIS_FOLDER = ''

df_nakeds = pd.read_pickle(DATAPATH.joinpath(THIS_FOLDER, 'df_nakeds.pkl'))

cols = ['symbol', 'strike', 'right', 'expiry', 'dte', 'conId', 'contract',
        'margin', 'bid', 'ask', 'iv', 'und_iv','price', 'lot', 'rom', 'sdMult', 'expRom', 'expPrice', 'qty']

df = df_nakeds[df_nakeds.price>0][cols].sort_values('rom', ascending=False)

In [9]:
# * REMOVE OPEN ORDERS FROM NAKEDS df
df_openords = get_openorders(MARKET)
df = df[~df.conId.isin(df_openords.conId)].reset_index(drop=True)

# ... build the naked SELL orders
contracts = df.contract.to_list()
orders = [LimitOrder(action='SELL', totalQuantity=abs(int(q)), lmtPrice=p) 
                        for q, p in zip(df.qty, df.expPrice)]

naked_cos = tuple((c, o) for c, o in zip(contracts, orders))

In [10]:
# Check if there are any naked open orders
df_openords

,conId,symbol,secType,expiry,strike,right,orderId,permId,action,totalQuantity,lmtPrice,status


## Place `naked` trades
* Make the following cell executable

In [11]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    ordered = place_orders(ib=ib, cos=naked_cos)

100%|██████████| 3/3 [00:02<00:00,  1.30it/s]Wall time: 2.35 s



## Clean up `naked` trades

# 3. `cover` trades

In [ ]:
# * IMPORTS
from ib_insync import LimitOrder
from support import get_openorders, place_orders

In [ ]:
# Extract files from Excel
covers_file = 'propose_covers.xlsx'
try:
    df_covers = pd.read_excel(DATAPATH.joinpath(covers_file), engine='openpyxl')
except FileNotFoundError:
    print(f'\n{covers_file} not found! ... Quitting !!')
    raise StopExecution

In [ ]:
# Get open orders
df_openords = get_openorders(MARKET)

# Prepare cover (contract, order)
if ~df_openords.empty: # Something exists in openorders
    # ... remove active openorders that conflict with df_covers
    df_covers1 = df_covers.set_index(['symbol', 'right', 'action']).\
                        join(df_openords[['symbol', 'right', 'action', 'orderId']].\
                        set_index(['symbol', 'right', 'action']))
    df_covers2 = df_covers1[df_covers1.orderId.isnull()].drop('orderId',1).reset_index()

    df_covers = df_covers2 # ready to return!


# ... build the naked SELL orders
covercts = [eval(c) for c in df_covers.contract]
# covercts = df_covers.contract.to_list()
coverords = [LimitOrder(action=act, totalQuantity=qty, lmtPrice=p) for act, qty, p in zip(df_covers.action, df_covers.qty, df_covers.expPrice)]
cover_cos = tuple((c, o) for c, o in zip(covercts, coverords))

## Place `cover` trades
* Make the following cell executable

## Cleanup `cover` trades

# 4. `orphan` trades

In [ ]:
# * IMPORTS
from ib_insync import LimitOrder
from support import get_openorders, place_orders

In [ ]:
# Extract files from Excel
orphans_file = 'propose_orphans.xlsx'
try:
    df_orphans = pd.read_excel(DATAPATH.joinpath(orphans_file))
except FileNotFoundError:
    print(f'\n{orphans_file} not found! ... Quitting !!')
    raise StopExecution

In [ ]:
# Get open orders
df_openords = get_openorders(MARKET)

# Prepare cover (contract, order)
if ~df_openords.empty: # Something exists in openorders
    # ... remove active openorders that conflict with df_orphans
    df_orphans1 = df_orphans.set_index(['symbol', 'right', 'action']).\
                        join(df_openords[['symbol', 'right', 'action', 'orderId']].\
                        set_index(['symbol', 'right', 'action']))
    df_orphans2 = df_orphans1[df_orphans1.orderId.isnull()].drop('orderId',1).reset_index()

    df_orphans = df_orphans2 # ready to return!
    
# ... build the naked SELL orders
df_orphans.contract = [eval(c) for c in df_orphans.contract]

## Prepare orders and contracts
orphancts = df_orphans.contract.to_list()
orphanords = [LimitOrder(action=act, totalQuantity=qty, lmtPrice=p) for act, qty, p in zip(df_orphans.action, df_orphans.qty, df_orphans.expPrice)]
orphan_cos = tuple((c, o) for c, o in zip(orphancts, orphanords))

## Place `orphan` trades
* make the following cell executable

In [ ]:
orphan_cos

# 5. Delete all `naked` sells
(in open orders)

In [ ]:
from support import quick_pf, get_openorders

In [ ]:
with IB().connect(HOST, PORT, CID) as ib:
    df_pf = quick_pf(ib)
    
    ib.reqAllOpenOrders()  # To kickstart collection of open orders
    ib.sleep(0.3)
    trades = ib.trades() # Get the trades
    
    stocks_in_pf = df_pf[df_pf.secType == 'STK'].symbol.to_list()
    
    orders = {t.order for t in trades 
              if t.contract.symbol not in stocks_in_pf # protects covered calls and puts
                if t.orderStatus.status == 'Submitted'
                   if t.order.action == 'SELL'}

BLK = 25
ords = list(orders)
o_blk = [ords[i:i+BLK] for i in range(0, len(ords), BLK)]